In [89]:
import pandas as pd
import numpy as np 
from itertools import permutations

In [90]:
#generate random data 
data = np.random.randint(10, size=(4, 2))
labels = ['A', 'B', 'C','D']
df = pd.DataFrame(data, columns=['X', 'Y'], index=labels)

#convert to a dictionary 
data_dict = {}
for idx, col in df.iterrows(): 
    data_dict[idx] = col.values
    
#data_dict   

In [91]:
#calculate the manhattan distance of our data values + put it into a n x n distance matrix 
def manhattan_dist(x, y):
    return sum(abs(a-b) for a,b in zip(x,y))

In [92]:
#put distances and labels into a dictionary
def build_tree():
    D = {}
    for label, val in data_dict.items():
        D[label] = {}
        for label2, val2 in data_dict.items():
            D[label][label2] = manhattan_dist(val, val2) 
    return D

In [93]:
tree = build_tree()
tree

{'A': {'A': 0, 'B': 3, 'C': 7, 'D': 4},
 'B': {'A': 3, 'B': 0, 'C': 6, 'D': 3},
 'C': {'A': 7, 'B': 6, 'C': 0, 'D': 3},
 'D': {'A': 4, 'B': 3, 'C': 3, 'D': 0}}

In [148]:
#visualize as n x n distance matrix         
dm = pd.DataFrame.from_dict(tree)
dm

,A,B,C,D
A,0,3,7,4
B,3,0,6,3
C,7,6,0,3
D,4,3,3,0


In [95]:
#check if the matrix is additive using four point condition
def is_additive(matrix, labels):
    i = labels[0]
    j = labels[1] 
    k = labels[2] 
    l = labels[3] 
    result = ''
    if ((matrix[i][j] + matrix[k][l] <= matrix[i][k] + matrix[j][l]) and 
    (matrix[i][k] + matrix[j][l] == matrix[i][l] + matrix[j][k])):
        result = "Additive"
    else:
        result = "Not additive"
    return result 

In [133]:
cols = 'ABCD'
labels = permutations(cols,4)
print([label for label in labels])

[('A', 'B', 'C', 'D'), ('A', 'B', 'D', 'C'), ('A', 'C', 'B', 'D'), ('A', 'C', 'D', 'B'), ('A', 'D', 'B', 'C'), ('A', 'D', 'C', 'B'), ('B', 'A', 'C', 'D'), ('B', 'A', 'D', 'C'), ('B', 'C', 'A', 'D'), ('B', 'C', 'D', 'A'), ('B', 'D', 'A', 'C'), ('B', 'D', 'C', 'A'), ('C', 'A', 'B', 'D'), ('C', 'A', 'D', 'B'), ('C', 'B', 'A', 'D'), ('C', 'B', 'D', 'A'), ('C', 'D', 'A', 'B'), ('C', 'D', 'B', 'A'), ('D', 'A', 'B', 'C'), ('D', 'A', 'C', 'B'), ('D', 'B', 'A', 'C'), ('D', 'B', 'C', 'A'), ('D', 'C', 'A', 'B'), ('D', 'C', 'B', 'A')]


In [140]:
def is_additive_full(dm):
    cols = 'ABCD'
    labels = permutations(cols,4)
    check = []
    for combs in labels:
        new_label = []
        for item in combs:
            new_label+=item
        check.append(is_additive(dm, new_label))
    return check
check

def additive_matrix(init_matrix, init_labels):
    add_flag = "Not additive" in is_additive_full(init_matrix)
    dm = None
    while add_flag:
        #generate random data 
        data = np.random.randint(10, size=(4, 2))
        print(data)
        labels = ['A', 'B', 'C','D']
        df = pd.DataFrame(data, columns=['X', 'Y'], index=labels)

        #convert to a dictionary 
        data_dict = {}
        for idx, col in df.iterrows(): 
            data_dict[idx] = col.values
        tree = build_tree()
        dm = pd.DataFrame.from_dict(tree)
        add_flag = "Not additive" in is_additive_full(dm)
        print(add_flag)
    return dm           

In [141]:
matrix = additive_matrix(dm, labels)
print(matrix)

[[2 2]
 [1 5]
 [0 2]
 [0 1]]
True
[[8 8]
 [3 4]
 [9 5]
 [7 5]]
True
[[3 0]
 [1 0]
 [5 6]
 [5 0]]
True
[[0 6]
 [6 1]
 [1 1]
 [3 4]]
True
[[2 9]
 [8 7]
 [3 3]
 [0 1]]
True
[[1 2]
 [1 6]
 [0 7]
 [6 5]]
True
[[7 6]
 [5 9]
 [9 5]
 [8 2]]
True
[[0 8]
 [1 3]
 [1 5]
 [5 4]]
True
[[5 0]
 [6 8]
 [0 9]
 [2 3]]
True
[[9 3]
 [5 8]
 [1 7]
 [0 8]]
True
[[9 9]
 [8 4]
 [9 6]
 [4 9]]
True
[[4 6]
 [8 6]
 [7 7]
 [9 0]]
True
[[3 2]
 [8 9]
 [4 8]
 [6 3]]
True
[[8 2]
 [2 6]
 [2 8]
 [3 6]]
True
[[0 1]
 [1 3]
 [9 1]
 [4 9]]
True
[[3 6]
 [1 4]
 [1 0]
 [3 7]]
True
[[0 9]
 [4 6]
 [2 3]
 [7 7]]
True
[[0 9]
 [5 7]
 [2 1]
 [9 2]]
True
[[3 3]
 [8 2]
 [0 1]
 [7 5]]
True
[[4 8]
 [4 1]
 [8 0]
 [0 8]]
True
[[0 1]
 [6 8]
 [0 5]
 [2 5]]
True
[[2 2]
 [5 1]
 [4 1]
 [2 4]]
True
[[7 5]
 [3 1]
 [9 6]
 [1 4]]
True
[[1 0]
 [5 6]
 [2 1]
 [9 3]]
True
[[2 5]
 [3 9]
 [6 2]
 [8 3]]
True
[[3 5]
 [4 8]
 [8 6]
 [1 6]]
True
[[7 6]
 [1 5]
 [8 0]
 [9 2]]
True
[[0 8]
 [7 5]
 [1 7]
 [0 4]]
True
[[2 1]
 [6 1]
 [2 2]
 [6 6]]
True
[[0 1]
 [0 5]


[[9 0]
 [3 6]
 [0 3]
 [7 3]]
True
[[2 7]
 [4 8]
 [3 4]
 [8 3]]
True
[[0 0]
 [7 4]
 [4 8]
 [8 2]]
True
[[8 4]
 [0 9]
 [1 4]
 [8 6]]
True
[[7 8]
 [2 2]
 [5 6]
 [6 0]]
True
[[8 0]
 [9 1]
 [2 5]
 [4 7]]
True
[[0 9]
 [7 6]
 [4 3]
 [3 9]]
True
[[8 5]
 [1 4]
 [8 2]
 [5 0]]
True
[[0 2]
 [9 0]
 [3 7]
 [3 9]]
True
[[6 7]
 [1 4]
 [6 5]
 [1 0]]
True
[[1 9]
 [7 2]
 [5 4]
 [6 9]]
True
[[7 9]
 [2 3]
 [9 5]
 [6 6]]
True
[[0 1]
 [8 7]
 [7 5]
 [6 5]]
True
[[5 3]
 [4 0]
 [1 1]
 [4 9]]
True
[[3 9]
 [9 7]
 [2 4]
 [1 0]]
True
[[1 5]
 [1 8]
 [7 4]
 [8 9]]
True
[[3 7]
 [7 5]
 [0 5]
 [8 1]]
True
[[9 1]
 [1 7]
 [0 4]
 [3 0]]
True
[[8 4]
 [6 5]
 [1 7]
 [4 5]]
True
[[0 8]
 [5 0]
 [3 4]
 [1 1]]
True
[[6 7]
 [1 8]
 [7 7]
 [1 1]]
True
[[7 0]
 [1 2]
 [0 5]
 [0 8]]
True
[[4 7]
 [6 3]
 [5 7]
 [0 1]]
True
[[8 4]
 [4 8]
 [6 5]
 [7 2]]
True
[[9 6]
 [3 9]
 [1 5]
 [2 5]]
True
[[2 1]
 [7 3]
 [0 5]
 [4 0]]
True
[[1 1]
 [0 4]
 [5 1]
 [9 8]]
True
[[3 9]
 [3 2]
 [2 1]
 [3 3]]
True
[[0 3]
 [0 1]
 [7 2]
 [5 0]]
True
[[2 5]
 [3 9]


True
[[6 4]
 [2 9]
 [7 7]
 [2 4]]
True
[[5 6]
 [7 8]
 [5 3]
 [8 4]]
True
[[3 0]
 [0 9]
 [9 6]
 [9 9]]
True
[[8 3]
 [6 9]
 [8 8]
 [3 1]]
True
[[6 6]
 [8 2]
 [0 9]
 [0 7]]
True
[[6 5]
 [5 9]
 [8 5]
 [6 4]]
True
[[4 5]
 [8 2]
 [5 9]
 [8 4]]
True
[[8 9]
 [9 9]
 [4 2]
 [2 4]]
True
[[7 4]
 [0 4]
 [8 5]
 [3 7]]
True
[[2 8]
 [9 8]
 [3 0]
 [7 8]]
True
[[7 7]
 [7 0]
 [4 5]
 [8 1]]
True
[[2 8]
 [2 0]
 [2 0]
 [2 9]]
True
[[8 8]
 [6 9]
 [1 0]
 [1 0]]
True
[[3 9]
 [0 2]
 [6 8]
 [6 5]]
True
[[9 5]
 [8 2]
 [7 0]
 [4 1]]
True
[[9 9]
 [7 0]
 [2 1]
 [3 0]]
True
[[0 8]
 [2 9]
 [8 4]
 [1 7]]
True
[[3 3]
 [3 7]
 [9 0]
 [5 1]]
True
[[1 1]
 [7 9]
 [4 1]
 [4 5]]
True
[[9 1]
 [2 5]
 [0 5]
 [9 6]]
True
[[9 9]
 [8 3]
 [8 5]
 [6 2]]
True
[[7 2]
 [3 1]
 [3 5]
 [1 5]]
True
[[2 3]
 [7 3]
 [2 3]
 [6 8]]
True
[[7 0]
 [7 8]
 [7 8]
 [7 2]]
True
[[8 6]
 [8 8]
 [8 9]
 [8 6]]
True
[[7 2]
 [4 0]
 [4 2]
 [3 9]]
True
[[1 0]
 [2 4]
 [6 7]
 [9 3]]
True
[[0 7]
 [5 3]
 [7 6]
 [2 0]]
True
[[7 8]
 [2 5]
 [6 6]
 [5 0]]
True
[[8 3]
 [

True
[[4 9]
 [4 7]
 [9 8]
 [8 6]]
True
[[5 1]
 [5 5]
 [9 8]
 [9 6]]
True
[[2 0]
 [3 8]
 [4 3]
 [1 7]]
True
[[6 2]
 [8 0]
 [8 1]
 [6 7]]
True
[[8 4]
 [7 8]
 [5 6]
 [7 0]]
True
[[1 5]
 [0 4]
 [2 5]
 [2 2]]
True
[[8 5]
 [0 2]
 [4 3]
 [6 4]]
True
[[4 7]
 [6 3]
 [1 3]
 [2 2]]
True
[[3 6]
 [5 1]
 [3 7]
 [5 5]]
True
[[3 4]
 [1 2]
 [3 8]
 [0 6]]
True
[[5 7]
 [8 8]
 [5 6]
 [8 8]]
True
[[6 9]
 [3 7]
 [6 1]
 [5 4]]
True
[[9 5]
 [4 0]
 [0 5]
 [5 7]]
True
[[4 5]
 [7 5]
 [9 8]
 [4 9]]
True
[[6 0]
 [7 7]
 [7 9]
 [9 3]]
True
[[8 4]
 [4 3]
 [6 1]
 [8 8]]
True
[[0 0]
 [8 6]
 [4 3]
 [0 6]]
True
[[2 6]
 [0 6]
 [7 1]
 [7 6]]
True
[[0 9]
 [1 2]
 [6 3]
 [4 3]]
True
[[4 8]
 [6 6]
 [6 0]
 [6 8]]
True
[[2 6]
 [2 8]
 [9 4]
 [7 5]]
True
[[1 7]
 [9 5]
 [9 3]
 [7 0]]
True
[[9 2]
 [0 8]
 [4 4]
 [5 5]]
True
[[3 5]
 [8 2]
 [0 1]
 [4 5]]
True
[[7 9]
 [9 4]
 [6 7]
 [5 3]]
True
[[9 0]
 [6 0]
 [6 9]
 [0 5]]
True
[[8 2]
 [2 8]
 [8 9]
 [9 8]]
True
[[2 7]
 [4 1]
 [6 0]
 [2 7]]
True
[[7 8]
 [0 2]
 [4 3]
 [2 4]]
True
[[9 3]
 [

[[9 9]
 [0 7]
 [0 1]
 [5 9]]
True
[[7 2]
 [8 3]
 [5 9]
 [5 4]]
True
[[5 2]
 [8 4]
 [5 2]
 [9 6]]
True
[[7 9]
 [4 2]
 [4 3]
 [7 1]]
True
[[1 9]
 [2 8]
 [7 6]
 [5 7]]
True
[[7 0]
 [9 1]
 [5 1]
 [7 4]]
True
[[2 2]
 [4 6]
 [4 8]
 [9 3]]
True
[[2 3]
 [5 9]
 [7 8]
 [5 7]]
True
[[3 3]
 [7 2]
 [3 8]
 [0 6]]
True
[[1 6]
 [3 0]
 [5 8]
 [2 7]]
True
[[6 5]
 [2 6]
 [2 8]
 [7 4]]
True
[[9 8]
 [3 3]
 [1 9]
 [5 6]]
True
[[0 6]
 [0 6]
 [8 1]
 [2 6]]
True
[[1 1]
 [7 9]
 [0 6]
 [8 2]]
True
[[9 5]
 [3 7]
 [8 0]
 [4 9]]
True
[[0 4]
 [8 8]
 [6 2]
 [8 8]]
True
[[6 9]
 [2 7]
 [0 4]
 [2 6]]
True
[[8 7]
 [6 2]
 [3 3]
 [8 6]]
True
[[7 0]
 [9 5]
 [5 9]
 [7 7]]
True
[[9 7]
 [6 4]
 [1 5]
 [3 5]]
True
[[4 1]
 [6 4]
 [3 6]
 [3 5]]
True
[[5 0]
 [9 1]
 [9 4]
 [7 4]]
True
[[2 4]
 [8 7]
 [9 1]
 [5 0]]
True
[[6 5]
 [3 3]
 [6 9]
 [1 0]]
True
[[3 2]
 [7 2]
 [0 6]
 [2 3]]
True
[[0 8]
 [7 0]
 [8 4]
 [3 4]]
True
[[2 7]
 [7 1]
 [4 9]
 [7 4]]
True
[[8 3]
 [5 5]
 [6 9]
 [7 7]]
True
[[8 5]
 [2 4]
 [3 3]
 [7 7]]
True
[[4 8]
 [3 5]


True
[[5 4]
 [9 5]
 [1 6]
 [0 6]]
True
[[5 0]
 [0 4]
 [8 4]
 [4 5]]
True
[[6 9]
 [5 8]
 [0 8]
 [1 9]]
True
[[8 3]
 [3 2]
 [9 2]
 [7 5]]
True
[[3 7]
 [9 5]
 [4 1]
 [3 0]]
True
[[8 6]
 [0 7]
 [5 1]
 [0 0]]
True
[[5 3]
 [3 4]
 [1 9]
 [1 4]]
True
[[8 7]
 [0 6]
 [2 3]
 [2 5]]
True
[[6 8]
 [5 9]
 [6 5]
 [9 3]]
True
[[7 4]
 [1 1]
 [1 3]
 [6 7]]
True
[[6 5]
 [4 3]
 [8 2]
 [0 0]]
True
[[9 4]
 [8 2]
 [0 0]
 [5 9]]
True
[[4 6]
 [7 8]
 [2 6]
 [8 0]]
True
[[0 5]
 [7 9]
 [0 5]
 [9 7]]
True
[[9 9]
 [3 8]
 [4 6]
 [8 5]]
True
[[1 4]
 [8 9]
 [4 0]
 [7 1]]
True
[[2 4]
 [3 1]
 [1 4]
 [3 9]]
True
[[5 4]
 [6 0]
 [4 8]
 [8 7]]
True
[[1 2]
 [9 1]
 [1 6]
 [0 6]]
True
[[1 6]
 [4 6]
 [7 4]
 [6 6]]
True
[[2 2]
 [9 9]
 [3 0]
 [0 9]]
True
[[6 7]
 [4 6]
 [2 4]
 [3 5]]
True
[[5 4]
 [8 8]
 [2 2]
 [6 6]]
True
[[0 1]
 [5 2]
 [9 6]
 [5 5]]
True
[[9 5]
 [1 6]
 [8 9]
 [5 8]]
True
[[1 2]
 [1 4]
 [0 4]
 [1 9]]
True
[[1 8]
 [8 6]
 [2 2]
 [7 0]]
True
[[9 9]
 [0 2]
 [9 0]
 [8 2]]
True
[[4 0]
 [0 7]
 [3 9]
 [4 3]]
True
[[8 6]
 [

[[1 1]
 [7 7]
 [9 9]
 [9 2]]
True
[[4 9]
 [0 3]
 [6 2]
 [2 6]]
True
[[6 5]
 [9 0]
 [4 1]
 [0 2]]
True
[[8 1]
 [9 4]
 [5 4]
 [9 4]]
True
[[7 5]
 [6 5]
 [9 4]
 [7 5]]
True
[[1 9]
 [2 9]
 [3 9]
 [8 0]]
True
[[0 8]
 [5 0]
 [9 8]
 [5 0]]
True
[[8 0]
 [7 4]
 [7 8]
 [2 2]]
True
[[5 8]
 [6 1]
 [1 3]
 [1 1]]
True
[[2 2]
 [1 8]
 [5 6]
 [9 2]]
True
[[3 3]
 [5 6]
 [2 7]
 [9 7]]
True
[[4 6]
 [3 2]
 [6 9]
 [6 8]]
True
[[6 8]
 [4 0]
 [1 6]
 [9 3]]
True
[[8 6]
 [0 7]
 [7 1]
 [5 4]]
True
[[7 0]
 [9 2]
 [8 5]
 [9 5]]
True
[[3 7]
 [9 3]
 [9 1]
 [2 0]]
True
[[7 1]
 [7 7]
 [2 7]
 [1 3]]
True
[[0 8]
 [8 3]
 [3 3]
 [2 8]]
True
[[1 2]
 [7 6]
 [1 1]
 [4 1]]
True
[[9 2]
 [9 5]
 [9 9]
 [7 9]]
True
[[0 0]
 [6 0]
 [3 0]
 [0 7]]
True
[[5 1]
 [5 0]
 [4 7]
 [5 2]]
True
[[3 3]
 [1 1]
 [2 6]
 [1 3]]
True
[[4 4]
 [2 9]
 [9 6]
 [5 0]]
True
[[4 1]
 [8 9]
 [8 1]
 [9 0]]
True
[[4 2]
 [0 0]
 [0 8]
 [6 0]]
True
[[1 5]
 [3 8]
 [7 0]
 [5 8]]
True
[[6 0]
 [2 4]
 [9 9]
 [6 4]]
True
[[9 1]
 [1 1]
 [6 7]
 [1 4]]
True
[[8 0]
 [3 0]


True
[[7 1]
 [7 2]
 [2 8]
 [7 5]]
True
[[7 5]
 [1 1]
 [9 8]
 [6 9]]
True
[[4 4]
 [3 8]
 [6 8]
 [7 5]]
True
[[3 5]
 [7 4]
 [2 8]
 [0 8]]
True
[[5 5]
 [4 9]
 [8 8]
 [1 9]]
True
[[4 8]
 [0 0]
 [9 5]
 [1 4]]
True
[[2 7]
 [1 7]
 [5 5]
 [1 3]]
True
[[9 0]
 [9 8]
 [1 2]
 [8 3]]
True
[[1 4]
 [9 4]
 [8 4]
 [6 2]]
True
[[6 6]
 [4 4]
 [1 0]
 [3 9]]
True
[[5 2]
 [7 6]
 [9 3]
 [2 5]]
True
[[8 3]
 [9 4]
 [2 3]
 [9 8]]
True
[[4 4]
 [9 7]
 [9 1]
 [8 5]]
True
[[7 9]
 [3 0]
 [8 1]
 [7 0]]
True
[[8 2]
 [8 5]
 [8 8]
 [2 8]]
True
[[6 9]
 [2 3]
 [2 5]
 [7 1]]
True
[[0 4]
 [1 6]
 [3 2]
 [5 7]]
True
[[1 7]
 [1 8]
 [2 8]
 [0 1]]
True
[[5 6]
 [0 8]
 [8 2]
 [8 6]]
True
[[8 3]
 [8 7]
 [6 1]
 [8 1]]
True
[[3 0]
 [7 7]
 [9 8]
 [9 9]]
True
[[6 7]
 [4 6]
 [5 2]
 [9 1]]
True
[[6 9]
 [3 5]
 [6 8]
 [3 4]]
True
[[8 0]
 [8 6]
 [9 3]
 [1 5]]
True
[[9 7]
 [2 5]
 [3 5]
 [7 5]]
True
[[7 1]
 [8 4]
 [0 4]
 [7 4]]
True
[[7 9]
 [4 7]
 [5 9]
 [1 3]]
True
[[8 4]
 [0 5]
 [1 9]
 [8 6]]
True
[[7 4]
 [7 5]
 [8 1]
 [0 3]]
True
[[3 6]
 [

KeyboardInterrupt: 

## Neigbor join

In [10]:
import copy
clusters = []
for cluster in list(dm):
    clusters.append(cluster)
cluster2idx = { name : idx for idx, name in enumerate(clusters) }
idx2cluster = copy.deepcopy(clusters)
m = copy.deepcopy(dm).to_dict()
Z = np.empty((0,4), float)
result = ""
print(clusters)
#membership = [set([cluster2idx[name]]) for name in clusters]

['A', 'B', 'C', 'D']


In [11]:
M, N = dm.shape
new_cluster_idx = M+1
while len(clusters) > 1:
    print(result)
    print(m)
    idx1 = None
    idx2 = None
    ui = 0
    uj = 0
    Sd = 100000
    for i in clusters:
        for j in clusters:
            if i != j:
                ui = sum(m[i][k] for k in clusters)
                uj = sum(m[j][k] for k in clusters)
                curr = (M - 2) * m[i][j] - ui - uj
                if Sd > curr:
                    Sd = curr
                    idx1 = cluster2idx[i]
                    idx2 = cluster2idx[j]
    
    Dij = m[idx2cluster[idx1]][idx2cluster[idx2]]
    new_cluster = "(" + idx2cluster[idx1] +"." + idx2cluster[idx2] + ")"
    result = new_cluster
    m[new_cluster] = {}
    new_cluster_idx += 1
    #membership.append(membership[cluster2idx[idx1]] | membership[cluster2idx[idx2]])
    #Z = np.append(Z, np.array([[idx1, idx2, Sd]]))
    
    m[idx2cluster[idx1]][new_cluster] = (m[idx2cluster[idx1]][idx2cluster[idx2]] + (1/(M-2)) * (ui - uj))/2
    m[idx2cluster[idx2]][new_cluster] = (m[idx2cluster[idx1]][idx2cluster[idx2]] + (1/(M-2)) * (uj - ui))/2
    
    #if idx2cluster[idx1] in m: del m[idx2cluster[idx1]]
    #if idx2cluster[idx2] in m: del m[idx2cluster[idx2]]
    
    clusters.remove(idx2cluster[idx1])
    clusters.remove(idx2cluster[idx2])
    
    m[new_cluster][new_cluster] = 0
    for c in clusters:
        m[c][new_cluster] = (m[c][idx2cluster[idx1]] + m[c][idx2cluster[idx2]] - Dij)/2
        m[new_cluster][c] = m[c][new_cluster]
        for k in list(m[c].keys()):
            if k == idx2cluster[idx1] or k == idx2cluster[idx2]:
                del m[c][k]
        
    del cluster2idx[idx2cluster[idx1]]
    del cluster2idx[idx2cluster[idx2]]
    if idx2cluster[idx1] in m: del m[idx2cluster[idx1]]
    if idx2cluster[idx2] in m: del m[idx2cluster[idx2]]
    
    clusters.append(new_cluster)
    cluster2idx[new_cluster] = len(idx2cluster)
    idx2cluster.append(new_cluster)
        
    #M -= 1
    #N -= 1


{'A': {'A': 0, 'B': 6, 'C': 6, 'D': 8}, 'B': {'A': 6, 'B': 0, 'C': 8, 'D': 2}, 'C': {'A': 6, 'B': 8, 'C': 0, 'D': 10}, 'D': {'A': 8, 'B': 2, 'C': 10, 'D': 0}}
(A.C)
{'B': {'B': 0, 'D': 2, '(A.C)': 4.0}, 'D': {'B': 2, 'D': 0, '(A.C)': 6.0}, '(A.C)': {'(A.C)': 0, 'B': 4.0, 'D': 6.0}}
(B.D)
{'(A.C)': {'(A.C)': 0, '(B.D)': 4.0}, '(B.D)': {'(B.D)': 0, '(A.C)': 4.0}}


In [12]:
print(result)

((A.C).(B.D))


In [1]:
##additive phlyogeny construction using degenerate triples

In [167]:

def additive_phylogeny(D,T):
    if D.shape == (2,2):
        T = {dm.index[0] : (dm.index[1], dm[dm.index[0]][dm.index[1]])}
        return T
    i,j,k,flag = degenerate_check(D)
    if



3